In [1]:
import pandas as pd
import numpy as np

In [2]:
games= pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/games.csv')
player_play=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/player_play.csv')
players=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/plays.csv')
track_w1=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_1.csv')
track_w2=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_2.csv')
track_w3=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_3.csv')
track_w4=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_4.csv')
track_w5=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_5.csv')
track_w6=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_6.csv')
track_w7=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_7.csv')
track_w8=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_8.csv')
track_w9=pd.read_csv('/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_9.csv')
